# Specutils Analysis

![Specutils: An Astropy Package for Spectroscopy](data/specutils_logo.png)


This notebook provides an overview of some of the spectral analysis capabilities of the Specutils Astropy coordinated package.  While this notebook is intended as an interactive introduction to specutils at the time of its writing, the canonical source of information for the package is the latest version's documentation: 

https://specutils.readthedocs.io

Note that the below assumes you have knowledge of the material in the [overview notebook](Specutils_overview.ipynb).  If this is not the case you may wish to review that notebook before proceding here.

## Imports

We start with some fundamental imports for working with specutils and simple visualization of spectra:

In [ ]:
import numpy as np

import astropy.units as u

import specutils
from specutils import Spectrum1D, SpectralRegion
specutils.__version__

In [ ]:
# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt


# for showing quantity units on axes automatically:
from astropy.visualization import quantity_support
quantity_support();

## Sample Spectrum and SNR

For use below, we also load the sample SDSS spectrum downloaded in the [overview notebook](Specutils_overview.ipynb).  See that notebook if you have not yet downloaded this spectrum.

In [ ]:
sdss_spec = Spectrum1D.read('data/sdss_spectrum.fits', format='SDSS-III/IV spec')
plt.step(sdss_spec.wavelength, sdss_spec.flux);

Because this example file already has uncertainties, it is straightforward to use one of the fundamental quantifications of a spectrum: the whole-spectrum signal-to-noise ratio:

In [ ]:
from specutils import analysis

analysis.snr(sdss_spec)

# Spectral Regions

Most analysis required on a spectrum requires specification of a part of the spectrum - e.g., a spectral line.  Because such regions may have value independent of a particular spectrum, they are represented as objects distrinct from a given spectrum object.  Below we outline a few ways such regions are specified.

In [ ]:
ha_region = SpectralRegion((6563-50)*u.AA, (6563+50)*u.AA)
ha_region

Regions can also be raw pixel values (although of course this is more applicable to a specific spectrum):

In [ ]:
pixel_region = SpectralRegion(2100*u.pixel, 2600*u.pixel)
pixel_region

Additionally, *multiple* regions can be in the same `SpectralRegion` object. This is useful for e.g. measuring multiple spectral features in one call:

In [ ]:
HI_wings_region = SpectralRegion([(1.4*u.GHz, 1.41*u.GHz), (1.43*u.GHz, 1.44*u.GHz)])
HI_wings_region

While regions are useful for a variety of analysis steps, fundamentally they can be used to extract sub-spectra from larger spectra:

In [ ]:
from specutils.manipulation import extract_region

subspec = extract_region(sdss_spec, pixel_region)
plt.step(subspec.wavelength, subspec.flux)

snr(subspec)

# Line Measurements

While line-fitting (detailed more below) is a good choice for high signal-to-noise spectra or when detailed kinematics are desired, more empirical measures are often used in the literature for noisier spectra or just simpler analysis procedures. Specutils provides a set of functions to provide these sorts of measurements, as well as similar summary statistics about spectral regions.  The [analysis part of the specutils documentation](https://specutils.readthedocs.io/en/latest/analysis.html) provides a full list and detailed examples of these, but here we demonstrate some example cases.

Note: these line measurements generally assume your spectrum is continuum-subtracted or continuum-normalized. Some spectral pipelines do this for you, but often this is not the case.  For our examples here we will do this step "by-eye", but for a more detailed discussion of continuum modeling, see the next section.  Based on the above plot we estimate a continuum level for the area of the SDSS spectrum around the H-alpha emission line, and use basic math to construct the continuum-normalized and continuum-subtracted spectra.

In [ ]:
# estimate a reasonable continuum-level estimate for the h-alpha area of the spectrum
sdss_continuum = 205*subspec.flux.unit

sdss_halpha_contsub = extract_region(sdss_spec, ha_region) - sdss_continuum

plt.axhline(0, c='k', ls=':')
plt.step(sdss_halpha_contsub.wavelength, sdss_halpha_contsub.flux)
plt.ylim(-50, 50)

With the continuum level identified, we can now make some measurements of the spectral lines that are apparent by eye - in particular we will focus on the H-alpha emission line. While there are techniques for identifying the line automatically (see the fitting section below), here we assume we are doing "quick-look" procedures where manual identification is possible. 

In the cell below, lill in the `<LOWER>` and `<UPPER>` values to make a spectral region that just encompasses the  H-alpha line (the middle of the three lines).

In [ ]:
line_region = SpectralRegion(<LOWER>*u.angstrom,
                             <UPPER>*u.angstrom)

plt.step(sdss_halpha_contsub.wavelength, sdss_halpha_contsub.flux)

yl1, yl2 = plt.ylim()
plt.fill_between([line_region.lower, line_region.upper], 
                 yl1, yl2, alpha=.2)
plt.ylim(yl1, yl2)

You can now call a variety of analysis functions on the continuum-subtracted spectrum to estimate various properties of the line:

In [ ]:
analysis.centroid(sdss_halpha_contsub, line_region)

In [ ]:
analysis.gaussian_fwhm(sdss_halpha_contsub, line_region)

In [ ]:
analysis.line_flux(sdss_halpha_contsub, line_region)

Equivalent width, being a continuum dependent property, can either be computed directly from the spectrum if the continuum level is given, or measured on a continuum-normalized spectrum. The latter is mainly useful if the continuum is non-uniform over the line being measured.

In [ ]:
analysis.equivalent_width(sdss_spec, sdss_continuum, regions=line_region)

In [ ]:
sdss_halpha_contnorm = sdss_spec / sdss_continuum
analysis.equivalent_width(sdss_halpha_contnorm, regions=line_region)

## Exercise

Load one of the spectrum datasets you made in the overview exercises into this notebook (i.e., your own dataset, a downloaded one, or the blackbody with an artificially added spectral feature).  Make a flux or width measurement of a line in that spectrum.  Anything odd?

# Continuum Subtraction

TODO

## Exercise

Using the spectrum from the previous exercise, first subtract a continuum, then re-do your measurement.  Is it better?

# Line-Fitting

TODO

## Exercise

Fit a spectral feature from your own spectrum using the fitting methods outlined above. Try the different line profile types (Gaussian, Lorentzian, or Voigt).  If you are using the blackbody spectrum (where you know the "true" answer for the spectral line), compare your answer to the true answer.